In [1]:
import pandas as pd
import numpy as np
import os

from tqdm import tqdm
from datetime import datetime

In [2]:
input_df = pd.read_csv("/opt/ml/input/data/FE_total_data.csv")
input_df = input_df.sort_values(by=["userID", "Timestamp"])
input_df.head(3)

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,dataset
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,1


# ElapsedTime: 한 문제당 푸는데 걸린 시간 추가

In [3]:
MAX_SECONDS = 300 # MAX_SECONDS를 넘겨서 푼 문제들은 딴 짓했다고 판단.

diffs = list()
time_stamps = input_df["Timestamp"]

for idx, (cur_time, next_time) in tqdm(enumerate(zip(time_stamps[:-1], time_stamps[1:])), total=len(time_stamps) - 1):
    
    cur_time = datetime.strptime(cur_time, "%Y-%m-%d %H:%M:%S")
    next_time = datetime.strptime(next_time, "%Y-%m-%d %H:%M:%S")
    
    # 현재 문제를 푸는데 걸린 시간 = 다음 문제를 푼 시간 - 현재 문제를 푼 시간
    time_diff = int((next_time - cur_time).total_seconds())
    
    # MAX_SECONDS를 넘기면 딴짓
    if time_diff > MAX_SECONDS or time_diff < 0: 
        diffs.append(0)
    else:
        diffs.append(time_diff)
    
# 가장 마지막 문제 풀이 시간은 모름
diffs.append(0)

input_df["elapsedTime"] = diffs

100%|██████████| 2526699/2526699 [00:49<00:00, 51323.91it/s]


In [4]:
# user가 test를 평균 몇초만에 해결했는지 저장
user_test_avg_time = dict()

for user in tqdm(input_df["userID"].unique()):
    
    # 유저의 시험지별 문제 평균 풀이 시간 계산
    x = input_df[input_df["userID"] == user].groupby("testId")["elapsedTime"].apply(list) #.apply(np.mean).apply(np.round).apply(int)
    
    # 유저의 시험지, 시험지의 문제 풀이시간들
    for test_id, times in zip(x.index, x.values):
        
        # 시험지의 문제 풀이시간들의 합이 0 : 너무 어려워서 모두 300초 이상 걸림 -> MAX_SECONDS + 1 (301초) 부여
        if np.sum(times) == 0:
            x[test_id] = MAX_SECONDS + 1
            
        # 시험지의 평균 문제풀이 시간계산
        else:
            x[test_id] = int(np.round(np.sum(times) / np.count_nonzero(times))) 
    user_test_avg_time[user] = x

100%|██████████| 7442/7442 [00:51<00:00, 144.15it/s]


In [5]:
# 시간이 0초인 것들 (알 수 없는 시간) 채우기
# Method1. 유저의 그 문제집 평균 시간으로 채운다.

for index, (elap_time, user_id, test_id) in tqdm(enumerate(zip(input_df["elapsedTime"], input_df["userID"], input_df["testId"])), total=len(input_df)):
    if elap_time == 0:
        input_df.at[index, "elapsedTime"] = user_test_avg_time[user_id][test_id]

100%|██████████| 2526700/2526700 [00:14<00:00, 177489.78it/s]


In [6]:
# 문제별 풀이시간 결과
input_df.head(15)

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,dataset,elapsedTime
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,3
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1,8
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,1,7
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,1,7
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,1,11
5,0,A060001007,A060000001,1,2020-03-24 00:17:47,7225,1,7
6,0,A060003001,A060000003,0,2020-03-26 05:52:03,7226,1,7
7,0,A060003002,A060000003,1,2020-03-26 05:52:10,7226,1,64
8,0,A060003003,A060000003,1,2020-03-26 05:53:14,7226,1,15
9,0,A060003004,A060000003,1,2020-03-26 05:53:29,7226,1,19


In [7]:
os.makedirs('/opt/ml/input/data/FE', exist_ok=True)
input_df.to_csv("/opt/ml/input/data/FE/elapsedTime.csv", index=False)